# Simple Tests

Some simple checks can be performed to test the performance and the behavior of the `EUNN` and the `EURNN` units.

## Imports

In [1]:
import torch
import numpy as np
import sys; sys.path.append('..')
from torch_eunn import EUNN

np.set_printoptions(precision=2, suppress=True)

## Test Unitarity

The action of a EUNNLayer should always be unitary.

In [2]:
%%time

# dimensionality of the cell
num_hidden = 50

# create new cell
cell = EUNN(num_hidden)

# get result of action of cell on identity matrix:
x = torch.stack([torch.eye(num_hidden, num_hidden), torch.zeros(num_hidden, num_hidden)], -1)
y = cell(x)
y = y[...,0].detach().numpy() + 1j*y[...,1].detach().numpy()

# check unitarity of result
print(np.abs(y@y.T.conj()))

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
CPU times: user 17.5 ms, sys: 448 µs, total: 18 ms
Wall time: 29.7 ms


We see that the operation of a `EUNN` is clearly unitary.

## Test Universality

Next we check if a full capacity cell can approximate any unitary matrix

In [3]:
%%time

# dimensionality of the cell
num_hidden = 10

# create new cell
cell = EUNN(num_hidden, num_hidden)

# create unitary matrix to approximate
U, _, _ = np.linalg.svd(np.random.randn(num_hidden,num_hidden) + 1j*np.random.randn(num_hidden,num_hidden))
U_torch = torch.stack([
    torch.tensor(np.real(U.T.conj()), dtype=torch.float32),
    torch.tensor(np.imag(U.T.conj()), dtype=torch.float32),
], -1)

# create the target
# the cell needs to be trained such that action of the cell on U.T.conj() yields the identity
I_torch = torch.stack([
    torch.eye(num_hidden),   
    torch.zeros((num_hidden,num_hidden)),
], -1)

# criterion & optimizer
lossfunc = torch.nn.MSELoss()
optimizer = torch.optim.Adam(cell.parameters(), lr=0.0020)

# training
for _ in range(10000):
    optimizer.zero_grad()
    I_approx = cell(U_torch)
    loss = lossfunc(I_approx, I_torch)
    loss.backward()
    optimizer.step()

result = I_approx[...,0].detach().numpy() + 1j*I_approx[...,1].detach().numpy()

print(abs(result)**2)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
CPU times: user 46.3 s, sys: 186 ms, total: 46.4 s
Wall time: 46 s


We see that we can approximate the matrix U.

## Test Speed

We can compare the execution speed of the `EUNN`, which should act like a unitary matrix to a normal complex matrix layer.

In [4]:
from torch_eunn import cm
class ComplexLayer(torch.nn.Module):
    def __init__(self, hidden_size):
        super(ComplexLayer, self).__init__()
        self.hidden_size = hidden_size
        self.W = torch.nn.Parameter(torch.randn(1, hidden_size, 2))
    def forward(self, x):
        return cm(x, self.W)

In [5]:
batch_size = 30
hidden_size = 30

# create layers
complex_layer = ComplexLayer(hidden_size)
unitary_layer = EUNN(hidden_size)
unitary_layer_cap2 = EUNN(hidden_size, capacity=2)

# create input vector
x = torch.randn(batch_size, hidden_size, 2)

# time speeds
%time y = complex_layer(x)
%time y = unitary_layer(x)
%time y = unitary_layer_cap2(x)

CPU times: user 183 µs, sys: 1 µs, total: 184 µs
Wall time: 190 µs
CPU times: user 4.66 ms, sys: 0 ns, total: 4.66 ms
Wall time: 4.31 ms
CPU times: user 814 µs, sys: 0 ns, total: 814 µs
Wall time: 751 µs


We see that the unitary EUNN implementation is still considerably slower than a normal complex multiplication. However, for capacity 2 networks (which are recommended to be used in recurrent neural networks), the difference is not big, while the potential benefits (no vanishing/exploding gradients) can be huge.